## Building Hidden Markov Models (*HMMs*) for speech recognition

We'll use *Gaussian HMMs* to model our data. *HMMs* are great tool for modeling time series data. As an audio signa is a time series signal, so the *HMMs* fit our needs. 

*HMMs* are popular because they can be trained automatically and are simple and computationally feasible to use. A *HMM* represent probability distrubutions over sequencs of observations.

In this document we use **hmmlearn** package built with Sphinx and **python_speech features** built for MFCC.


In [111]:
import os
import warnings

import numpy as np
from scipy.io import wavfile 
from hmmlearn import hmm
from python_speech_features import mfcc

In [112]:
# class to handle HMM processing
class HMMTrainer(object):  
    '''
    Parameters
    ----------
    
    n_components: parameter defines the number of hidden states
    cov_type: defines the type of covariance in transition matrix
    n_iter: indicates the number of iterations for traning
    
    Choice of parameters depends on the data. 
    '''
    def __init__(self, model_name='GaussianHMM', n_components=3, covariance_type="diag",
                 init_params="cm", params="cmt", n_iter=1000):
        
        # initialize
        self.model_name = model_name
        self.n_components = n_components
        self.covariance_type = covariance_type
        self.init_params = init_params
        self.params = params
        self.n_iter = n_iter
        self.models = []


        # define model
        if self.model_name == 'GaussianHMM':
            self.model = hmm.GaussianHMM(n_components=self.n_components,
                                         covariance_type=self.covariance_type,
                                         init_params=self.init_params, 
                                         params=self.params,
                                         n_iter=self.n_iter)
        else:
            raise TypeError('Invalid model type')
            
            
    # train data is 2D aray, where each frow is k-dimensions
    def train(self, X):
        
        # ingonre divisin by 0
        np.seterr(all='ignore')
        self.models.append(self.model.fit(X))
        
    # run the model on input data and get score
    def get_score(self, input_data):
        return self.model.score(input_data)

We need train data to build our speech recognizer. We will use the database available at [here](https://code.google.com/archive/p/hmm-speech-recognition/downloads). This data set contains seven different words, where each word has 15 audio files associated with it. We'll build an *HMM* model for each class by training our model on given dataset. Then after build model, given new input file, we need to run all the models on this file and pick the one with the best score. 

In [113]:
if __name__ == '__main__':

    # init the variables of all HMM models
    hmm_models = []
    
    # define train folder
    dataset = 'train'
    input_folder = 'data\{}'.format(dataset)
    
    
    # get path of input files
    try:
        audiofiles = os.listdir(input_folder)
    except FileNotFoundError:
        assert False, "Folder not found"

    # pars the input directory that contains audio files
    for dirname in audiofiles:
        # get the name of the subfolder
        subfolder = os.path.join(input_folder, dirname)
        
        if not os.path.isdir(subfolder):
            continue    
        
        # extract the label
        label = subfolder[subfolder.rfind('\\')+1:]

        
        # initialize intput variables and labels
        X = np.array([])
        y_words = []
        
        # iterate through the audio files
        for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')]:
            # Read the input file
            filepath = os.path.join(subfolder, filename)
            sampling_freq, audio = wavfile.read(filepath)
            
            # Extract MFCC features
            mfcc_features = mfcc(audio, sampling_freq)

            # Append to the variable X
            if len(X) == 0:
                X = mfcc_features
            else:
                X = np.append(X, mfcc_features, axis=0)
            
            # Append the label
            y_words.append(label)
            
        
        # Train and save HMM model
#         print ('X.shape ='+ str(X.shape))
        hmm_trainer = HMMTrainer()
        hmm_trainer.train(X)
        hmm_models.append((hmm_trainer, label))
        hmm_trainer = None

C:\Users\Anar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Anar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Anar\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\A

After extracted features from all the files, train and save the HMM model.
As HMM is a generative model for unsupervised learning, we don't need labels
to build HMM modes for each class. We explitly assume that seperate HMM models
will be built for each class

In [114]:
    # define test folder
    dataset = 'test'
    input_folder = 'data\{}'.format(dataset)
    
    # test folder
    test_files = []

    # get path of test files
    try:
        audiofiles = os.listdir(input_folder)
    except FileNotFoundError:
        assert False, "Folder not found"

    # pars the input directory that contains audio files
    for dirname in audiofiles:

        # get the name of the subfolder
        subfolder = os.path.join(input_folder, dirname)

        if not os.path.isdir(subfolder):
                continue

        # iterate through the audio files
        for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')]:
            # read the input file

            filepath = os.path.join(subfolder, filename)
            test_files.append(filepath)      

In [115]:
    test_files

['data\\test\\apple\\apple15.wav',
 'data\\test\\banana\\banana15.wav',
 'data\\test\\kiwi\\kiwi15.wav',
 'data\\test\\lime\\lime15.wav',
 'data\\test\\orange\\orange15.wav',
 'data\\test\\peach\\peach15.wav',
 'data\\test\\pineapple\\pineapple15.wav']

In [119]:
    # Classify input data
    for input_file in test_files:
        

     

        sampling_freq, audio = wavfile.read(input_file)
        
      

        # Extract MFCC features
        mfcc_features = mfcc(audio, sampling_freq)

        # Define variables
        max_score = hmm_models[0][0].get_score(mfcc_features)
        output_label = None

        # Iterate through all HMM models and pick 
        # the one with the highest score
        for item in hmm_models:
            hmm_model, label = item
            score = hmm_model.get_score(mfcc_features)
            if score >= max_score:
                max_score = score
                output_label = label
                
        print ("\nTrue: " + input_file[input_file.find('\\')+6:input_file.rfind('\\')])
        print ("Predicted:"+ str(output_label))


True: apple
Predicted:apple

True: banana
Predicted:banana

True: kiwi
Predicted:kiwi

True: lime
Predicted:lime

True: orange
Predicted:orange

True: peach
Predicted:peach

True: pineapple
Predicted:pineapple
